# Main Idea

Denote the average of the row $r_{avg}$. The operation we want to perform would be

$$ (C - R_{avg}) w$$

where $R_{avg}$ is the matrix obtained by stacking $r_{avg}$.

Navie implementation would be directly computing $C_m = C - R_{avg}$ and then right multiply it by the vector $w$.

A more efficient approach would be to distribute the multiplication. $C w$ is easier to compute because of sparsity, and $R_{avg}w$ is simply a vector full of $r_{avg} \cdot w$.

**Speed Gain**: since the operations are in the order of $O(NK)$ and the density $d$ represents how much fewer operations we need, we expect the speed gain to be $O(NKd)$

In [11]:
import numpy as np
import time
from scipy.sparse import csr_matrix
import scipy as sp

In [2]:
lmbda = 0.1
K = 10000
N = 10000
dims = [N, K]

Generate sparse matrix using poisson distribution

In [3]:
# in dense matrix format, no performance improvement
C = np.random.poisson(lmbda, dims)

# in sparse matrix format, certain operations should be faster
C_sparse = csr_matrix(C)

# average of the rows of C
r_avg = np.mean(C, axis=0)

# w, in this case we use the one vector
w = np.ones(K)

In [10]:
print(np.count_nonzero(C_sparse.toarray()))

9511194


In [14]:
print("The sparsity is around", np.count_nonzero(C_sparse.toarray()) / N / K)

The sparsity is around 0.09511194


## Naive implementation:

In [15]:
# The jupyter-notebook's magic commands, %t expr,
# will print the amount of time needed to evaluate expr
%time (C - r_avg) @ w

CPU times: user 574 ms, sys: 1.16 s, total: 1.73 s
Wall time: 2.14 s


array([  8.6502,   7.6502, -17.3498, ..., -38.3498,  26.6502, -49.3498])

## Efficient implementation but on dense matrix:

In [16]:
%time C @ w - np.dot(r_avg, w)

CPU times: user 321 ms, sys: 316 ms, total: 636 ms
Wall time: 602 ms


array([  8.6502,   7.6502, -17.3498, ..., -38.3498,  26.6502, -49.3498])

## Efficient implementation but on sparse matrix:

In [17]:
# %time C_sparse @ v - v.sum() * c_avg
%time C_sparse @ w - np.dot(r_avg, w)

CPU times: user 36.6 ms, sys: 41.9 ms, total: 78.5 ms
Wall time: 79.1 ms


array([  8.6502,   7.6502, -17.3498, ..., -38.3498,  26.6502, -49.3498])

## Conclusion

We can see that the naive implementation is the slowest because of 
1. centering (subtracting from $c_{avg}$ takes $O(NK)$ in runtime)
2. multiplication (also $O(NK)$ in runtime)

In the efficient implementation on dense matrix, by distributing the multiplication, we can simplify the operation on $\left[ c_{avg} ... c_{avg} \right] v$, but $Cv$ is still slow because of dense matrix.

In the efficient implementation on the sparse matrix, now both operations have been **optimized**.